In [1]:
import pydss
import numpy as np
from bokeh.io import output_notebook, show
from bokeh.plotting import figure
from bokeh.layouts import gridplot
from bokeh.models import ColumnDataSource, HoverTool, CustomJS
output_notebook()

Loading BokehJS ...

In [36]:
E = 250.0 # phase voltage rms
i_dc_av = 10.0 # dc current
v_c= 565.0
L_s = 1.0e-3
omega_e = 2*np.pi*50.0
sqrt2 = np.sqrt(2.0)
sqrt3 = np.sqrt(3.0)
pi = np.pi
r_dc= 0.01
L_dc = 5.0e-3
i_dc_av = 1.0
Dt = 100.0e-6

for it in range(100):
    di_dc_av = (3*sqrt3/pi*sqrt2*E - (3/pi*omega_e*L_s+r_dc)*i_dc_av - v_c)/(2*L_s+L_dc)
    mu = np.arccos(1-np.sqrt(2.0/3.0)*omega_e*L_s/(E)*i_dc_av)

    
    i_dc_av += Dt*di_dc_av


i_qs_com = -2.0*sqrt3/pi*i_dc_av*( np.sin(mu-5*pi/6) + np.sin(5*pi/6)) - 3.0/pi*sqrt2*E/(omega_e*L_s)*(np.cos(mu)-1) +3/(4*pi)*sqrt2*E/(omega_e*L_s)*(np.cos(2*mu)-1)                             
i_ds_com = -2.0*sqrt3/pi*i_dc_av*( np.cos(5*pi/6) - np.cos(mu-5*pi/6)) - 3.0/pi*sqrt2*E/(omega_e*L_s)*(np.sin(mu)) +3/(4*pi)*sqrt2*E/(omega_e*L_s)*(np.sin(2.0*mu) + 2*mu)                             
i_qs_cond = -2.0*sqrt3/pi*i_dc_av*( np.sin(7*pi/6) - np.sin(mu + 5*pi/6))
i_ds_cond = -2.0*sqrt3/pi*i_dc_av*( np.cos(mu + 5*pi/6) - np.cos(7*pi/6))
    

print(i_qs_com)
print(i_ds_com)
print(i_qs_cond)
print(i_ds_cond)



4.87838308209
1.49975550651
20.7252547532
2.28078203531


In [24]:
sys1.bokeh_tools()

p = figure(width=600, height=200,
           title='LV Network', 
           x_range=[-10,130], y_range=[-20, 20])

# lines:
source = ColumnDataSource(sys1.line_data)
lin = p.multi_line(source=source, xs='x_s', ys='y_s', color="red", alpha=0.5, line_width=5)

# buses:
source = ColumnDataSource(sys1.bus_data)
cr = p.circle(source=source, x='x', y='y', size=15, color="navy", alpha=0.5)

p.add_tools(HoverTool(renderers=[lin], tooltips=sys1.line_tooltip))
p.add_tools(HoverTool(renderers=[cr], tooltips=sys1.bus_tooltip))
show(p)


NameError: name 'sys1' is not defined

In [6]:
import numpy as np
from bokeh.io import output_notebook, show
from bokeh.models import ColumnDataSource
from bokeh.io import push_notebook
from bokeh.layouts import gridplot
from bokeh.plotting import figure
from bokeh.charts import Bar 
import pandas as pd
output_notebook()

sys1.pq_3pn = s_0
sys1.update()
sys1.pf_eval()
sys1.get_v()
sys1.get_i()

plt_voltages = figure(width=600, height=300,x_range = [-10,130], y_range = [180,250])
source = ColumnDataSource(sys1.bus_data)
cr = plt_voltages.circle(source=source, x='x', y='v_an', size=15, color="red", alpha=0.5)
plt_voltages.circle(source=source, x='x', y='v_bn', size=15, color="green", alpha=0.5)
plt_voltages.circle(source=source, x='x', y='v_cn', size=15, color="blue", alpha=0.5)
plt_voltages.line([-300, 350],[231*1.05,231*1.05], color='red', line_width=5)
plt_voltages.line([-300, 350],[231*0.90,231*0.90], color='blue', line_width=5)
#plt_voltages.add_tools(HoverTool(renderers=[cr], tooltips=sys1.bus_tooltip))

s = sys1.system.V_known * np.conj(sys1.system.I_unknown)
p = s.real/1000

d = {'bus':['N1']*4+['N2']*4,'node':['a','b','c','n']*2,'p':list(p[:,0]), 'pos':range(8)}
#source_powers = pd.DataFrame(d)
source_powers = ColumnDataSource(d)


#plt_powers = Bar(source_powers, values='p',label='bus',  group='node', 
#        title="Avg MPG by ORIGIN, stacked by CYL") #  legend='top_left'
plt_powers = figure(width=600, height=300, y_range = [-100,150])
plt_powers.circle(source=source_powers, x='pos', y='p', size=15, color="green", alpha=0.5)

def update(load_factor=1.0):

    sys1.pq_3pn = load_factor*s_0
    sys1.update()
    sys1.pf_eval()
    sys1.get_v()
    sys1.get_i()
    sys1.bokeh_tools()
    s = sys1.system.V_known * np.conj(sys1.system.I_unknown)
    p = s.real/1000

    source.data = sys1.bus_data
    source_powers.data= {'bus':['N1']*4+['N2']*4,'node':['a','b','c','n']*2,'p':list(p[:,0]), 'pos':range(8)}

    push_notebook()

p_grid = gridplot([[plt_voltages], [plt_powers]])
show(p_grid,notebook_handle=True)


#show(p, notebook_handle=True)


Loading BokehJS ...

In [7]:
from ipywidgets import interact
interact(update, load_factor=(-1,1.2, 0.01))

<function __main__.update>

In [174]:
from imp import reload
import pydss
pydss = reload(pydss)

In [175]:
sys1 = pydss.pydss('bench_3bus.json')
sys1.system.run_eval()

In [176]:
fig = figure(width=600, height=300)
T = sys1.system.T
Y = sys1.system.out
X = sys1.system.X
for it in range(8):
    colors = ['red','green','blue','black']
    
    fig.line(T[:,0],np.abs(Y[it+0,:]), color=colors[it%4], line_width=1)
show(fig)

In [146]:
sys1.get_v()
sys1.get_i()
sys1.bokeh_tools()

p = figure(width=600, height=200,
           title='LV Network', 
           x_range=[-10,130], y_range=[-20, 20])

# lines:
source = ColumnDataSource(sys1.line_data)
lin = p.multi_line(source=source, xs='x_s', ys='y_s', color="red", alpha=0.5, line_width=5)

# buses:
source = ColumnDataSource(sys1.bus_data)
cr = p.circle(source=source, x='x', y='y', size=15, color="navy", alpha=0.5)

p.add_tools(HoverTool(renderers=[lin], tooltips=sys1.line_tooltip))
p.add_tools(HoverTool(renderers=[cr], tooltips=sys1.bus_tooltip))
show(p)

In [10]:
figp = figure(width=600, height=300)
for it in range(6):
    colors = ['red','green','blue','black']
    P=sys1.system.out_cplx[it+72,:].real/1000
    figp.line(T[:,0],P, color=colors[it%3], line_width=1)
show(figp)

In [11]:
sys1.system.out_cplx[0,:].real

array([ 0.,  0.,  0., ...,  0.,  0.,  0.])

In [177]:
fig = figure(width=600, height=300)
for it in range(12):
    colors = ['red','green','blue','black']
    
    fig.line(T[:,0],np.abs(sys1.system.out_cplx[it,:])/230, color=colors[it%4], line_width=1)
show(fig)